# Compound Interest & Periodic Contributions: A Mathematical Deep Dive

This notebook explores the growth of a portfolio using a **Fixed-Point Analysis** approach to solve non-homogeneous recurrence relations. 

## 1. Mathematical Framework

We model the portfolio value $V_n$ after $n$ periods with the following variables:
* $P$: Initial principal ($V_0$)
* $c$: Periodic contribution (made at the start of each period)
* $r$: Interest rate per period
* $R$: Growth multiplier ($1 + r$)

### The Recurrence Relation
Using an **annuity due** model (deposits made before interest is applied):
$$V_{n+1} = (V_n + c)R$$

### Fixed-Point Analysis
To solve this, we find the fixed point $A$ where $A = (A + c)R$:
$$A = AR + cR \implies A(1-R) = cR$$
$$A = \frac{cR}{1-R} = -\frac{cR}{r}$$

By shifting the sequence, we get a geometric progression $V_n - A = (P - A)R^n$. Substituting $A$ back in gives the closed-form solution:
$$V_n = P(1+r)^n + c(1+r)\frac{(1+r)^n - 1}{r}$$

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, IntSlider

def calculate_portfolio(P, c, r_annual, years, freq='Monthly'):
    # Adjust parameters based on frequency
    periods_per_year = {'Monthly': 12, 'Quarterly': 4, 'Annually': 1}[freq]
    n_total = years * periods_per_year
    r = (r_annual / 100) / periods_per_year
    R = 1 + r
    
    # Calculate time steps
    periods = np.arange(n_total + 1)
    
    # Closed-form solution components
    principal_growth = P * (R**periods)
    contribution_growth = c * R * ((R**periods - 1) / r) if r > 0 else c * periods
    
    total_value = principal_growth + contribution_growth
    
    return periods / periods_per_year, total_value, principal_growth, contribution_growth

## 2. Interactive Visualization
Use the sliders below to see how the **fixed-point offset** and the power of compounding affect the portfolio over time.

In [ ]:
def update_plot(P, c, r_annual, years):
    years_arr, total, principal, contrib = calculate_portfolio(P, c, r_annual, years)
    
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(go.Scatter(x=years_arr, y=principal, name='Principal Growth', 
                             fill='tozeroy', line=dict(color='#2ecc71')))
    fig.add_trace(go.Scatter(x=years_arr, y=total, name='Total Value (inc. Contributions)', 
                             fill='tonexty', line=dict(color='#3498db')))
    
    fig.update_layout(
        title=f'Portfolio Projection: ${total[-1]:,.2f}',
        xaxis_title='Years',
        yaxis_title='Value ($)',
        hovermode='x unified',
        template='plotly_white'
    )
    fig.show()

interact(update_plot, 
         P=IntSlider(min=0, max=100000, step=1000, value=10000, description='Principal ($)'),
         c=IntSlider(min=0, max=5000, step=100, value=500, description='Monthly Contrib ($)'),
         r_annual=FloatSlider(min=0, max=15, step=0.1, value=7.0, description='Annual % Rate'),
         years=IntSlider(min=1, max=40, step=1, value=20, description='Time (Years)'));

### Summary of the Model
The visualization highlights the **non-linear** nature of the growth. 

| Component | Impact |
| :--- | :--- |
| **Initial Principal ($P$)** | Subjected to $R^n$, dominates in the early years if $P$ is large. |
| **Contributions ($c$)** | Scaled by the geometric series $\frac{R^n-1}{r}$, dominates long-term results. |
| **Fixed Point ($A$)** | Represents the theoretical 'debt' level that would be balanced by interest. |